In [35]:
import os
import math
import random as rn

import cv2 as cv
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import keras

from keras.models import Model, Sequential
from keras.layers import *
from keras import optimizers
from keras.preprocessing import image
from keras.applications import mobilenet
from keras.utils.np_utils import to_categorical
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input, decode_predictions

import tensorflow.keras.backend as K
# to set the image order
K.set_image_data_format('channels_first')

In [36]:
SEED = 12345
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
rn.seed(SEED)

In [37]:
COLOR_MODE = "grayscale"
EPOCHS = 50
BATCH_SIZE = 512

In [38]:
def LeNet_5():
    model = Sequential()
    model.add(Conv2D(32,(5,5),strides=(1,1),input_shape=(32,32,1),padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(64,(5,5),strides=(1,1),padding='same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10,activation='softmax'))
    return model

In [39]:
# Load MobileNet model
model = LeNet_5()
opt = optimizers.SGD(lr=1e-2, momentum=0.9),
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
# Fetch the dataset directly
dataset = tfds.image_classification.Cifar10()

In [41]:
# Download the data, prepare it, and write it to disk
dataset.download_and_prepare()

In [42]:
# Describe the dataset with DatasetInfo
C = dataset.info.features['label'].num_classes
Ntrain = dataset.info.splits['train'].num_examples
Ntest = dataset.info.splits['test'].num_examples
Nbatch = BATCH_SIZE
assert C == 10
assert Ntrain == 50000
assert Ntest == 10000

In [43]:
# Load data from disk as tf.data.Datasets
datasets = dataset.as_dataset()
train_dataset, validation_dataset = datasets['train'], datasets['test']
assert isinstance(train_dataset, tf.data.Dataset)
assert isinstance(validation_dataset, tf.data.Dataset)

In [44]:
def dataset_generator(dataset, batch_size=BATCH_SIZE, num_classes=10, is_training=False, color_mode="grayscale"):
  images = np.zeros((batch_size, 32, 32, 1))
  labels = np.zeros((batch_size, num_classes))
  while True:
    count = 0 
    for sample in tfds.as_numpy(dataset):
      image = sample["image"]
      label = sample["label"]

      if color_mode == "grayscale":
        # rgb to grayscale
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
      else: # keep only one channel in RGB
        channel_index = list('BGR').index(color_mode.upper())
        image = image[:,:,channel_index]

      images[count%batch_size] = vgg16.preprocess_input(np.expand_dims(cv.resize(image, (32, 32)), -1))
      labels[count%batch_size] = np.expand_dims(to_categorical(label, num_classes=num_classes), 0)
      
      count += 1
      if (count%batch_size == 0):
        yield images, labels

In [45]:
# Infer on Cifar10
labels = np.zeros((Ntest))
pred_labels = np.zeros((Ntest, C))
pred_labels_new = np.zeros((Ntest, C))

# Train on Cifar10
checkpoint_path = "LeNet_5/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
os.makedirs(checkpoint_dir, exist_ok=True)

cp_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    monitor="val_loss",
    save_weights_only=True,
    save_best_only=True,
    mode='min',
)
    
csv_logger = keras.callbacks.CSVLogger('LeNet_5_training.csv')

model.fit_generator(dataset_generator(train_dataset, batch_size=Nbatch, is_training=True, color_mode=COLOR_MODE),
                        steps_per_epoch= Ntrain // Nbatch,
                        epochs = EPOCHS,
                        validation_data = dataset_generator(validation_dataset, batch_size=Nbatch, color_mode=COLOR_MODE),
                        validation_steps = Ntest // Nbatch,
                        verbose = 1,
                        callbacks = [cp_callback, csv_logger])

IndexError: index 1 is out of bounds for axis 2 with size 1

In [ ]:
checkpoint_path = "Mobilenet\cp-0020.ckpt"
model.load_weights(checkpoint_path)
score = model.evaluate_generator(
    dataset_generator(validation_dataset,batch_size=Nbatch,color_mode=COLOR_MODE),
    steps= Ntest // Nbatch,
    verbose=1,)

print("Evaluation Result of new Model on cifar100: " + str(score))

In [ ]:
model.save("model_{}_{:04d}.h5".format(COLOR_MODE, int(score[-1]*10000)))